In [1]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_community.graphs import Neo4jGraph
# from neo4j_graphrag import Text2Graph
from langchain_neo4j import Neo4jVector
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_neo4j import Neo4jGraph
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from langchain_neo4j import GraphCypherQAChain
# from langchain.graphs.neo4j_graph import Neo4jGraph
from dotenv import load_dotenv

In [19]:
from dotenv import load_dotenv

In [21]:
os.getenv("NGROK_URL")

'https://dereistic-ta-unrobustly.ngrok-free.dev'

In [3]:
template = """Question: {question}

Answer: Let's think step by step."""

In [4]:
prompt = ChatPromptTemplate.from_template(template)

In [9]:
model = OllamaLLM(model="gpt-oss:20b-cloud")

chain = prompt | model

In [6]:
# from neo4j import GraphDatabase
graph = Neo4jGraph(
    url=os.getenv("URL"),
    username=os.getenv("USERNAME"),
    password=os.getenv("PASSWORD")
)

In [10]:
chain_cypher = GraphCypherQAChain.from_llm(llm=model, graph=graph, verbose=True,allow_dangerous_requests = True)

## Not Cool

In [12]:
import ollama

In [13]:
prompt = f"Can you read anything from this {chain_cypher}"

In [ ]:
response = ollama.generate(model='gpt-oss:20b', prompt=prompt)

## USing the colab version of ollama

In [22]:
from ollama import Client

client = Client(host=os.getenv("NGROK_URL"))

response = client.chat(
    model="ministral-3:14b",
    messages=[{"role": "user", "content": "Hello"}]
)

print(response["message"]["content"])

ResponseError: The endpoint dereistic-ta-unrobustly.ngrok-free.dev is offline.

ERR_NGROK_3200
 (status code: 404)

## Asking questions based on the knowledge graph.

In [16]:
question = "Who is the CTO of Moon Finance?"
result = chain_cypher.invoke({"query": question})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[r:WORKS_AT]->(c:Company)
WHERE c.name = "Moon Finance" AND r.role = "CTO"
RETURN p.name;
Full Context:
[{'p.name': 'Soumil Binani'}]

> Finished chain.


In [17]:
result

{'query': 'Who is the CTO of Moon Finance?', 'result': 'Soumil Binani.'}

In [15]:
result['result']

'Aum, Soumil Binani.'

In [38]:
print(result["result"])

I don't know the answer.


## Inserting new memories :

In [18]:
new_memory = "Albert Einstien had a girlfriend named Sofia"
documents = [Document(page_content=new_memory)]

In [24]:
graph_transformer = LLMGraphTransformer(llm=model)
# graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)
# graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [27]:
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

AttributeError: 'list' object has no attribute 'get'

In [26]:
graph_documents[0].nodes  # list of nodes
graph_documents[0].relationships  # list of relationships

TypeError: 'coroutine' object is not subscriptable

In [34]:
await update_graph_in_neo4j(graph_documents)

In [35]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Sofia', type='Person', properties={}), Node(id='Albert Einstien', type='Person', properties={})]
Relationships:[Relationship(source=Node(id='Albert Einstien', type='Person', properties={}), target=Node(id='Sofia', type='Person', properties={}), type='HAS_GIRLFRIEND', properties={})]


In [ ]:
# Aiyo KG hatt naa jaye puraka pura:
summary = driver.execute_query(cypher_query_manual, database_="neo4j").summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))